# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 48](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=48)*

If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the URL you used to get the error. 

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "YESTERDAY"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

verbose = "true"  # TODO change to false before push, else true

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

verbose = (verbose == "true") 


In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,                
    limit=5000,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

DBG get_records endpoint='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports'
DBG get_records qstr: is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241110&max_day_obs=20241111
DBG get_records len(page)=2 len(recs)=0
DBG get_records-2 len(page)=2 len(recs)=2
DBG get_records-2 len(self.records)=2 status={'endpoint_url': 'https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241110&max_day_obs=20241111', 'number_of_records': 2, 'error': None}
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241110&max_day_obs=20241111
DBG get_exposures len(page)=0 len(recs)=0
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LATISS&order_by=

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-10 to 2024-11-11 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-DRAFT)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: AuxTel

```

```

Telescope Status: 

*Authors: *

### Telescope: Simonyi

```
In the morning, there was a power outage, and by 8:10 AM, we switched to backup power. This caused the OSS to go into fault, the PS to turn off, alarms on the chillers, and required a restart of compressor #2. While bringing up the drives and PXI for the hexapods and rotator, we cleared the interlock from the chillers, reset the PS, and restored the OSS. 
We ran M2hexapod warm-ups, enabling the hexapod under test while keeping the other disabled. 
Around 13:20h CLT, Jaques noticed that compressor #2 was not working properly, and HPs air compressors were not reaching pressures over 117 PSI, so we restarted it, and after the restart, M1M3 pressure reached 125 PSI but just for couple of minutes, then it was oscillating between 116 and 117 PSI, so we decided to turn on the LaserTracker first. To do this, we had to turn off the OSS, LOTO on, M1M3 standby, laser tracker on, then put back to M1M3 Enabled, LOTO off, and OSS on, then M1M3 pressure reached 125 PSI again.
We then ran the CamHex warmup and once this finished, we started the HP breakaway test. M1M3 HP breakaway test done with EUI to run all HPs at the same time.
We ran the bump test after this. FA 235 P, 238 Y, 305 Y failed bump test but when we re-ran individually, we saw 235 and 238 are slightly overshooting and 305 passed. We raised the mirror. FA 215 that failed yesterday was kept disabled.
During the bump test, we unparked the TMA to elevation 80 using EUI and conducted the TMA and dome parallel tests while taking sample acquisition images in the r and u bands with lights off. Brian adjusted the default system speed to 5%. An alignment test (T88) was performed, and the laser tracker was turned off. 
To avoid the Sun, the dome was moved to 140°. Earthquake happened and MTMount and M1M3 went into fault, with M1M3 Force controller X Moment Limit error. We recovered MTMount, M1M3, M1M3TS, and MTRotator. Clearing interlocks were needed for M1M3. HVAC was turned off, VEC04 on, we moved dome to az=220 so that we can reach the dome cabinet. Then the shutter and mirror covers were opened. After opening the shutter we moved dome away from sun again az=110. We ran track_target to check it’s working since we recovered many systems, with same az but to el=60. Twilight flats in the U band began at 20:25h CLT (~-2.5° Sun’s altitude), followed by darks, and an AOS test (BLOCK-T249, T248) was performed. During scheduled script run (resume.py), we had bad gateway issue (OBS-614, 623). Re-resuming 3rd time worked. We stopped the script to focus better, re-ran the script. During this, we got a azimuth drive failure with the MTDome and exitFault didn’t clear the error. Tiago recycled MTDome, reset cRIO, and re-home and defined the zeroAz there to recover the dome. While dome is being fixed, we took images and ran some test that could be done with fixed dome position (still tracking). We are back to on-sky but missed the good timing for T248. Moved onto BLOCK-Txx with late night shift.
```

Telescope Status: 

*Authors: HyeYun Park, Gonzalo Aravena, Elana Urbach, Guillem Megias Homar, Andrew Rasmussen, Kate Napier, Carlos Morales*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-10**

,(times in UTC),
Moon Rise,2024-11-09 16:37:10.226,
Moon Set,2024-11-10 06:08:59.597,
Moon Illumination,60%,(% illuminated)
Astronomical Twilight (morning),2024-11-10 08:11:41.944,(-18 degrees)
Astronomical Twilight (evening),2024-11-10 00:41:49.376,(-18 degrees)
Solar Midnight,2024-11-10 04:26:45.656,
Nautical Twilight (morning),2024-11-10 08:44:04.416,(-12 degrees)
Nautical Twilight (evening),2024-11-10 00:09:26.878,(-12 degrees)
Civil Twilight (morning),2024-11-10 09:15:02.510,(-6 degrees)
Civil Twilight (evening),2024-11-09 23:38:28.725,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.


In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCam
Total Night,7:29:52
Total Exposure,1:21:59
Slew time(1),0:00:00
Readout time(2),0:09:09
Time loss to fault,NA
Time loss to weather,NA
Idle time,6:07:53
Number of exposures,239
Mean readout time,0.000639
Number of slews(1),0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
- Observing Operations - OBS
- TODO: better tickets using Jira API

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241110&max_day_obs=20241111) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

- Should we replace zero with empty string?
- TODO: Do not show empty tables. ie no table when "0 total exposures"
- TODO: Add link to exposures detail when A field value (e.g. observation_reason=science) is clicked on.  The detail should include all exposures counted int the total for that field value.

In [13]:
# Exposure Report
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        if df.empty:
            continue
        print(field_name.title().replace('_',' '))
        display(df)

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 239 total exposures**

Observation Type


,good,questionable,junk,unknown,total
acq,0,0,0,94,94
bias,0,0,0,10,10
cwfs,0,0,0,81,81
dark,0,0,0,46,46
science,0,0,0,8,8


Observation Reason


,good,questionable,junk,unknown,total
checking,0,0,0,6,6
comcam-usdf-checkout,0,0,0,10,10
comcam_twilight_flat,0,0,0,87,87
extra,0,0,0,3,3
extra_alignment_sweep_m2rx,0,0,0,7,7
extra_alignment_sweep_m2ry,0,0,0,17,17
extra_best_m2_ry,0,0,0,1,1
extra_sitcom-826,0,0,0,12,12
infocus,0,0,0,3,3
infocus_alignment_sweep_m2rx,0,0,0,7,7


Science Program


,good,questionable,junk,unknown,total
BLOCK-T215,0,0,0,87,87
BLOCK-T246,0,0,0,10,10
BLOCK-T248,0,0,0,17,17
BLOCK-T249,0,0,0,11,11
BLOCK-T250,0,0,0,6,6
BLOCK-T279,0,0,0,98,98
BLOCK-T92,0,0,0,2,2
PP-SURVEY,0,0,0,8,8


**LSSTCam: 0 total exposures**

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [14]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-10 11:07:15.143285** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Keeping MTM1M3 in Enabled  so it doesn't fault</pre>


- **2024-11-10 10:56:40.425009** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Hexapod shutdown</pre>


- **2024-11-10 10:02:18.321268** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We try to send the rotator to zero position but failed salIndex: 101390</pre>


- **2024-11-10 09:38:04.028897** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>WE repeat the procedure to cool down LSST in the first level the temperature was -15, now is around -35</pre>


- **2024-11-10 09:13:16.168956** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>we home axes in the EUI, now we are moving the MTMount to inicial position</pre>


...

- **2024-11-10 16:58:32.938245** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Starting Warm-up for M2 Hexapod and CamHexapod</pre>


- **2024-11-10 15:58:34.237203** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>ComCam usdf checkout (BLOCK-T246) done</pre>


- **2024-11-10 15:51:10.530313** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Cleared the interlock of chillers, turning OSS on.</pre>


- **2024-11-10 15:23:52.530045** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We had PS and OSS off for TMA, due to power outage.
Turning back PS on</pre>


- **2024-11-10 15:23:52.522869** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Powering up both rotator and hexapods</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [15]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-10 12:00:00**.
- min_dayobs='2024-11-10', max_dayobs='2024-11-11'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [16]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,2,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241110&max_day_obs=20241111
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241110&max_day_obs=20241111
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241110&max_day_obs=20241111
exposurelog/exposures.LSSTComCam,239,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241110&max_day_obs=20241111
exposurelog/exposures.LSSTCam,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241110&max_day_obs=20241111
exposurelog/messages,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241110&max_day_obs=20241111
narrativelog/messages,150,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-10T00%3A00%3A00&max_date_added=2024-11-11T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [17]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [18]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [2]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241110&max_day_obs=20241111

None
- Got 2 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [150]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-10T00%3A00%3A00&max_date_added=2024-11-11T00%3A00%3A00

None
- Got 150 records.  


In [19]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

### According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument.
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [20]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

<font style="color:lightblue;background-color:None;">No exposurelog records found 2024-11-10 to 2024-11-11. </font>

Used [API Data](https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241110&max_day_obs=20241111)

## Finale

In [21]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-11 22:31:53.852033
